In [148]:
#Cragamos librerias
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [149]:
#Cargar desde un archivo .csv sin indice
df = pd.read_csv('Albany.csv')
df.head(5)

,Unnamed: 0.1,Unnamed: 0,listing_url,last_scraped,source,name,neighborhood_overview,picture_url,host_url,host_name,...,beds,minimum_nights,maximum_nights,availability_365,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,reviews_per_month
0,0,0,https://www.airbnb.com/rooms/2992450,2024-11-05,city scrape,Luxury 2 bedroom apartment,nice,https://a0.muscache.com/pictures/44627226/0e72...,https://www.airbnb.com/users/show/4621559,Kenneth,...,2.0,1.7,1125,219,9.0,4.90,4.90,4.90,4.80,0.07
1,1,1,https://www.airbnb.com/rooms/3820211,2024-11-05,previous scrape,Restored Precinct in Center Sq. w/Parking,"Great restaurants, architecture, walking, peop...",https://a0.muscache.com/pictures/prohost-api/H...,https://www.airbnb.com/users/show/19648678,Terra,...,1.0,2.0,1125,363,30.6,4.75,4.87,4.81,4.82,2.46
2,2,2,https://www.airbnb.com/rooms/5651579,2024-11-05,city scrape,Large studio apt by Capital Center & ESP@,The neighborhood is very eclectic. We have a v...,https://a0.muscache.com/pictures/b3fc42f3-6e5e...,https://www.airbnb.com/users/show/29288920,Gregg,...,1.0,2.0,45,21,30.6,4.51,4.45,4.87,4.76,3.20
3,3,3,https://www.airbnb.com/rooms/6623339,2024-11-05,city scrape,Center Sq. Loft in Converted Precinct w/ Parking,"Located in Albany's finest urban neighborhood,...",https://a0.muscache.com/pictures/prohost-api/H...,https://www.airbnb.com/users/show/19648678,Terra,...,1.0,2.0,1125,339,30.6,4.73,4.71,4.70,4.80,2.86
4,4,4,https://www.airbnb.com/rooms/9005989,2024-11-05,city scrape,"Studio in The heart of Center SQ, in Albany NY","There are many shops, restaurants, bars, museu...",https://a0.muscache.com/pictures/d242a77e-437c...,https://www.airbnb.com/users/show/17766924,Sugey,...,2.0,1.0,1125,357,30.6,4.79,4.83,4.93,4.87,5.96


**Convertimos a numericos**

In [150]:
#df['host_response_time'] = df['host_response_time'].map({'a few days or more':1, 'within an hour':2,'within a few hours':3, 'within a day':4})
df['host_is_superhost'] = df['host_is_superhost'].map({'t': 1, 'f': 0})
df["host_response_rate"] = df["host_response_rate"].str.rstrip('%').astype(float) / 100
#df['room_type'] = df['room_type'].map({'Entire home/apt': 1,'Private room': 2,'Shared room': 3,'Hotel room': 4})
#df["host_acceptance_rate"] = df["host_acceptance_rate"].str.rstrip('%').astype(float) / 100
df['host_verifications'] = df['host_verifications'].map({"['email', 'phone', 'work_email']":1, "['email', 'phone']":2, "['phone']": 3,"['phone', 'work_email']": 4, "['email']": 5,"[]": 6,"['email', 'work_email']": 7,"['work_email']": 8})
df['host_has_profile_pic'] = df['host_has_profile_pic'].map({'t': 1, 'f': 0})
df['host_identity_verified'] = df['host_identity_verified'].map({'t': 1, 'f': 0})
df['has_availability'] = df['has_availability'].map({'t': 1, 'f': 0})
df['instant_bookable'] = df['instant_bookable'].map({'t': 1, 'f': 0})


In [151]:
df['property_type'] = df['property_type'].replace({
    'Entire rental unit': 1,
    'Entire condo': 2,
    'Private room in rental unit': 3,
    'Entire home': 4,
    'Private room in condo': 5,
    'Entire townhouse': 6,
    'Private room in home': 7,
    'Entire villa': 8,
    'Entire loft': 9,
    'Private room in villa': 10,
    'Entire serviced apartment': 11,
    'Private room in townhouse': 12,
    'Room in hotel': 13,
    'Entire guesthouse': 14,
    'Private room in bed and breakfast': 15,
    'Room in aparthotel': 16,
    'Private room in guest suite': 17,
    'Private room in casa particular': 18,
    'Shared room in hotel': 19,
    'Private room in guesthouse': 20,
    'Tiny home': 21,
    'Shared room in hostel': 22,
    'Private room in serviced apartment': 23,
    'Private room in boat': 24,
    'Houseboat': 25,
    'Entire guest suite': 26,
    'Boat': 27,
    'Shared room in rental unit': 28,
    'Shared room in condo': 29,
    'Private room in loft': 30,
    'Private room': 31,
    'Private room in bungalow': 32,
    'Private room in hostel': 33,
    'Entire bungalow': 34,
    'Entire vacation home': 35,
    'Entire cabin': 36,
    'Private room in shipping container': 37,
    'Room in hostel': 38,
    'Camper/RV': 39,
    'Entire place': 40,
    'Barn': 41,
    'Casa particular': 42,
    'Entire cottage': 43,
    'Private room in hut': 44,
    'Shared room in bungalow': 45,
    'Private room in tiny home': 46,
    'Shared room in bed and breakfast': 47,
    'Private room in vacation home': 48,
    'Hut': 49,
    'Private room in barn': 50,
    'Minsu': 51,
    'Shipping container': 52,
    'Private room in cabin': 53,
    'Tower': 54
})


C:\Users\ady_b\AppData\Local\Temp\ipykernel_27584\2122643265.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['property_type'] = df['property_type'].replace({


**Reemplazar valores**

_host_acceptance_rate_

In [290]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['host_acceptance_rate'])
unico

array([0, 1])

In [153]:
df["host_acceptance_rate"] = df["host_acceptance_rate"].str.rstrip('%').astype(float) / 100

In [154]:
#Convierto una variable a dicotómica
df['host_acceptance_rate'] = df['host_acceptance_rate'].apply(
    lambda x: 1 if isinstance(x, (int, float)) and x > 0.80 else (0 if isinstance(x, (int, float)) and x <= 0.80 else 'Sin respuesta')
)

df

,Unnamed: 0.1,Unnamed: 0,listing_url,last_scraped,source,name,neighborhood_overview,picture_url,host_url,host_name,...,beds,minimum_nights,maximum_nights,availability_365,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,reviews_per_month
0,0,0,https://www.airbnb.com/rooms/2992450,2024-11-05,city scrape,Luxury 2 bedroom apartment,nice,https://a0.muscache.com/pictures/44627226/0e72...,https://www.airbnb.com/users/show/4621559,Kenneth,...,2.0,1.7,1125,219,9.0,4.90,4.90,4.90,4.80,0.07
1,1,1,https://www.airbnb.com/rooms/3820211,2024-11-05,previous scrape,Restored Precinct in Center Sq. w/Parking,"Great restaurants, architecture, walking, peop...",https://a0.muscache.com/pictures/prohost-api/H...,https://www.airbnb.com/users/show/19648678,Terra,...,1.0,2.0,1125,363,30.6,4.75,4.87,4.81,4.82,2.46
2,2,2,https://www.airbnb.com/rooms/5651579,2024-11-05,city scrape,Large studio apt by Capital Center & ESP@,The neighborhood is very eclectic. We have a v...,https://a0.muscache.com/pictures/b3fc42f3-6e5e...,https://www.airbnb.com/users/show/29288920,Gregg,...,1.0,2.0,45,21,30.6,4.51,4.45,4.87,4.76,3.20
3,3,3,https://www.airbnb.com/rooms/6623339,2024-11-05,city scrape,Center Sq. Loft in Converted Precinct w/ Parking,"Located in Albany's finest urban neighborhood,...",https://a0.muscache.com/pictures/prohost-api/H...,https://www.airbnb.com/users/show/19648678,Terra,...,1.0,2.0,1125,339,30.6,4.73,4.71,4.70,4.80,2.86
4,4,4,https://www.airbnb.com/rooms/9005989,2024-11-05,city scrape,"Studio in The heart of Center SQ, in Albany NY","There are many shops, restaurants, bars, museu...",https://a0.muscache.com/pictures/d242a77e-437c...,https://www.airbnb.com/users/show/17766924,Sugey,...,2.0,1.0,1125,357,30.6,4.79,4.83,4.93,4.87,5.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,416,416,https://www.airbnb.com/rooms/1277527400878120655,2024-11-05,city scrape,Experience Historic Albany Full Throttle!,nice,https://a0.muscache.com/pictures/hosting/Hosti...,https://www.airbnb.com/users/show/382970529,Thomas,...,1.4,2.0,365,365,0.0,4.90,4.90,5.00,4.80,2.20
417,417,417,https://www.airbnb.com/rooms/1277717926531215616,2024-11-05,city scrape,Large Center Sq Luxury Retreat.,nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/65760,Anthony,...,3.0,1.7,1125,363,0.0,4.90,4.90,5.00,4.80,2.20
418,418,418,https://www.airbnb.com/rooms/1279011643102137684,2024-11-05,city scrape,nice cool stays,nice,https://a0.muscache.com/pictures/hosting/Hosti...,https://www.airbnb.com/users/show/539608284,Joyhefsiba,...,1.0,4.0,365,365,0.0,4.90,4.90,5.00,4.80,2.20
419,419,419,https://www.airbnb.com/rooms/1279781671588026640,2024-11-05,city scrape,Spacious & Quiet Townhouse.,nice,https://a0.muscache.com/pictures/hosting/Hosti...,https://www.airbnb.com/users/show/528617680,Paola,...,1.0,1.0,365,365,1.0,4.90,4.90,4.90,4.80,1.00


_room_type_

In [155]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['room_type'])
unico

array(['Entire home/apt', 'Private room', 'Shared room'], dtype=object)

In [156]:
#Convierto una variable a dicotómica
df['room_type'] = df['room_type'].replace(["Shared room", "Private room"], 0)
df['room_type'] = df['room_type'].replace(["Entire home/apt", "Hotel room"], 1)
df

C:\Users\ady_b\AppData\Local\Temp\ipykernel_27584\508382164.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['room_type'] = df['room_type'].replace(["Entire home/apt", "Hotel room"], 1)


,Unnamed: 0.1,Unnamed: 0,listing_url,last_scraped,source,name,neighborhood_overview,picture_url,host_url,host_name,...,beds,minimum_nights,maximum_nights,availability_365,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,reviews_per_month
0,0,0,https://www.airbnb.com/rooms/2992450,2024-11-05,city scrape,Luxury 2 bedroom apartment,nice,https://a0.muscache.com/pictures/44627226/0e72...,https://www.airbnb.com/users/show/4621559,Kenneth,...,2.0,1.7,1125,219,9.0,4.90,4.90,4.90,4.80,0.07
1,1,1,https://www.airbnb.com/rooms/3820211,2024-11-05,previous scrape,Restored Precinct in Center Sq. w/Parking,"Great restaurants, architecture, walking, peop...",https://a0.muscache.com/pictures/prohost-api/H...,https://www.airbnb.com/users/show/19648678,Terra,...,1.0,2.0,1125,363,30.6,4.75,4.87,4.81,4.82,2.46
2,2,2,https://www.airbnb.com/rooms/5651579,2024-11-05,city scrape,Large studio apt by Capital Center & ESP@,The neighborhood is very eclectic. We have a v...,https://a0.muscache.com/pictures/b3fc42f3-6e5e...,https://www.airbnb.com/users/show/29288920,Gregg,...,1.0,2.0,45,21,30.6,4.51,4.45,4.87,4.76,3.20
3,3,3,https://www.airbnb.com/rooms/6623339,2024-11-05,city scrape,Center Sq. Loft in Converted Precinct w/ Parking,"Located in Albany's finest urban neighborhood,...",https://a0.muscache.com/pictures/prohost-api/H...,https://www.airbnb.com/users/show/19648678,Terra,...,1.0,2.0,1125,339,30.6,4.73,4.71,4.70,4.80,2.86
4,4,4,https://www.airbnb.com/rooms/9005989,2024-11-05,city scrape,"Studio in The heart of Center SQ, in Albany NY","There are many shops, restaurants, bars, museu...",https://a0.muscache.com/pictures/d242a77e-437c...,https://www.airbnb.com/users/show/17766924,Sugey,...,2.0,1.0,1125,357,30.6,4.79,4.83,4.93,4.87,5.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,416,416,https://www.airbnb.com/rooms/1277527400878120655,2024-11-05,city scrape,Experience Historic Albany Full Throttle!,nice,https://a0.muscache.com/pictures/hosting/Hosti...,https://www.airbnb.com/users/show/382970529,Thomas,...,1.4,2.0,365,365,0.0,4.90,4.90,5.00,4.80,2.20
417,417,417,https://www.airbnb.com/rooms/1277717926531215616,2024-11-05,city scrape,Large Center Sq Luxury Retreat.,nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/65760,Anthony,...,3.0,1.7,1125,363,0.0,4.90,4.90,5.00,4.80,2.20
418,418,418,https://www.airbnb.com/rooms/1279011643102137684,2024-11-05,city scrape,nice cool stays,nice,https://a0.muscache.com/pictures/hosting/Hosti...,https://www.airbnb.com/users/show/539608284,Joyhefsiba,...,1.0,4.0,365,365,0.0,4.90,4.90,5.00,4.80,2.20
419,419,419,https://www.airbnb.com/rooms/1279781671588026640,2024-11-05,city scrape,Spacious & Quiet Townhouse.,nice,https://a0.muscache.com/pictures/hosting/Hosti...,https://www.airbnb.com/users/show/528617680,Paola,...,1.0,1.0,365,365,1.0,4.90,4.90,4.90,4.80,1.00


_availability_365_

In [157]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['availability_365'])
unico

array([  0,   1,   3,   6,   7,   9,  12,  14,  15,  16,  18,  20,  21,
        30,  33,  40,  45,  46,  49,  52,  55,  57,  61,  64,  65,  66,
        67,  69,  70,  72,  75,  76,  77,  78,  79,  80,  81,  82,  83,
        84,  85,  86,  87,  88,  89,  90,  92,  93,  96, 104, 105, 111,
       118, 121, 123, 128, 130, 132, 139, 140, 147, 152, 154, 156, 157,
       158, 161, 163, 164, 166, 167, 168, 169, 171, 173, 174, 177, 179,
       180, 185, 186, 189, 192, 194, 198, 199, 201, 202, 204, 205, 206,
       211, 213, 214, 216, 218, 219, 222, 228, 231, 233, 234, 237, 238,
       240, 243, 244, 247, 248, 249, 250, 251, 252, 253, 254, 257, 258,
       260, 263, 264, 265, 267, 268, 269, 270, 276, 277, 278, 280, 293,
       296, 299, 304, 306, 307, 308, 309, 313, 314, 315, 322, 323, 325,
       327, 328, 330, 331, 332, 333, 334, 336, 337, 338, 339, 340, 341,
       342, 343, 345, 346, 347, 348, 349, 350, 351, 353, 354, 355, 356,
       357, 358, 359, 360, 361, 362, 363, 364, 365])

In [158]:
#Convierto una variable a dicotómica
df['availability_365'] = df['availability_365'].apply(
    lambda x: 1 if x == 365 else (0 if isinstance(x, (int, float)) and x < 365 else 'Sin respuesta')
)

df

,Unnamed: 0.1,Unnamed: 0,listing_url,last_scraped,source,name,neighborhood_overview,picture_url,host_url,host_name,...,beds,minimum_nights,maximum_nights,availability_365,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,reviews_per_month
0,0,0,https://www.airbnb.com/rooms/2992450,2024-11-05,city scrape,Luxury 2 bedroom apartment,nice,https://a0.muscache.com/pictures/44627226/0e72...,https://www.airbnb.com/users/show/4621559,Kenneth,...,2.0,1.7,1125,0,9.0,4.90,4.90,4.90,4.80,0.07
1,1,1,https://www.airbnb.com/rooms/3820211,2024-11-05,previous scrape,Restored Precinct in Center Sq. w/Parking,"Great restaurants, architecture, walking, peop...",https://a0.muscache.com/pictures/prohost-api/H...,https://www.airbnb.com/users/show/19648678,Terra,...,1.0,2.0,1125,0,30.6,4.75,4.87,4.81,4.82,2.46
2,2,2,https://www.airbnb.com/rooms/5651579,2024-11-05,city scrape,Large studio apt by Capital Center & ESP@,The neighborhood is very eclectic. We have a v...,https://a0.muscache.com/pictures/b3fc42f3-6e5e...,https://www.airbnb.com/users/show/29288920,Gregg,...,1.0,2.0,45,0,30.6,4.51,4.45,4.87,4.76,3.20
3,3,3,https://www.airbnb.com/rooms/6623339,2024-11-05,city scrape,Center Sq. Loft in Converted Precinct w/ Parking,"Located in Albany's finest urban neighborhood,...",https://a0.muscache.com/pictures/prohost-api/H...,https://www.airbnb.com/users/show/19648678,Terra,...,1.0,2.0,1125,0,30.6,4.73,4.71,4.70,4.80,2.86
4,4,4,https://www.airbnb.com/rooms/9005989,2024-11-05,city scrape,"Studio in The heart of Center SQ, in Albany NY","There are many shops, restaurants, bars, museu...",https://a0.muscache.com/pictures/d242a77e-437c...,https://www.airbnb.com/users/show/17766924,Sugey,...,2.0,1.0,1125,0,30.6,4.79,4.83,4.93,4.87,5.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,416,416,https://www.airbnb.com/rooms/1277527400878120655,2024-11-05,city scrape,Experience Historic Albany Full Throttle!,nice,https://a0.muscache.com/pictures/hosting/Hosti...,https://www.airbnb.com/users/show/382970529,Thomas,...,1.4,2.0,365,1,0.0,4.90,4.90,5.00,4.80,2.20
417,417,417,https://www.airbnb.com/rooms/1277717926531215616,2024-11-05,city scrape,Large Center Sq Luxury Retreat.,nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/65760,Anthony,...,3.0,1.7,1125,0,0.0,4.90,4.90,5.00,4.80,2.20
418,418,418,https://www.airbnb.com/rooms/1279011643102137684,2024-11-05,city scrape,nice cool stays,nice,https://a0.muscache.com/pictures/hosting/Hosti...,https://www.airbnb.com/users/show/539608284,Joyhefsiba,...,1.0,4.0,365,1,0.0,4.90,4.90,5.00,4.80,2.20
419,419,419,https://www.airbnb.com/rooms/1279781671588026640,2024-11-05,city scrape,Spacious & Quiet Townhouse.,nice,https://a0.muscache.com/pictures/hosting/Hosti...,https://www.airbnb.com/users/show/528617680,Paola,...,1.0,1.0,365,1,1.0,4.90,4.90,4.90,4.80,1.00


_host_response_time_

In [159]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['host_response_time'])
unico

array(['a few days or more', 'unknown', 'within a day',
       'within a few hours', 'within an hour'], dtype=object)

In [160]:
df['host_response_time']=df['host_response_time'].replace(["within a few hours", "within a day", "a few days or more", "unknown"], 0)
df['host_response_time']=df['host_response_time'].replace(['within an hour'], 1)
df

C:\Users\ady_b\AppData\Local\Temp\ipykernel_27584\2343497725.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['host_response_time']=df['host_response_time'].replace(['within an hour'], 1)


,Unnamed: 0.1,Unnamed: 0,listing_url,last_scraped,source,name,neighborhood_overview,picture_url,host_url,host_name,...,beds,minimum_nights,maximum_nights,availability_365,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,reviews_per_month
0,0,0,https://www.airbnb.com/rooms/2992450,2024-11-05,city scrape,Luxury 2 bedroom apartment,nice,https://a0.muscache.com/pictures/44627226/0e72...,https://www.airbnb.com/users/show/4621559,Kenneth,...,2.0,1.7,1125,0,9.0,4.90,4.90,4.90,4.80,0.07
1,1,1,https://www.airbnb.com/rooms/3820211,2024-11-05,previous scrape,Restored Precinct in Center Sq. w/Parking,"Great restaurants, architecture, walking, peop...",https://a0.muscache.com/pictures/prohost-api/H...,https://www.airbnb.com/users/show/19648678,Terra,...,1.0,2.0,1125,0,30.6,4.75,4.87,4.81,4.82,2.46
2,2,2,https://www.airbnb.com/rooms/5651579,2024-11-05,city scrape,Large studio apt by Capital Center & ESP@,The neighborhood is very eclectic. We have a v...,https://a0.muscache.com/pictures/b3fc42f3-6e5e...,https://www.airbnb.com/users/show/29288920,Gregg,...,1.0,2.0,45,0,30.6,4.51,4.45,4.87,4.76,3.20
3,3,3,https://www.airbnb.com/rooms/6623339,2024-11-05,city scrape,Center Sq. Loft in Converted Precinct w/ Parking,"Located in Albany's finest urban neighborhood,...",https://a0.muscache.com/pictures/prohost-api/H...,https://www.airbnb.com/users/show/19648678,Terra,...,1.0,2.0,1125,0,30.6,4.73,4.71,4.70,4.80,2.86
4,4,4,https://www.airbnb.com/rooms/9005989,2024-11-05,city scrape,"Studio in The heart of Center SQ, in Albany NY","There are many shops, restaurants, bars, museu...",https://a0.muscache.com/pictures/d242a77e-437c...,https://www.airbnb.com/users/show/17766924,Sugey,...,2.0,1.0,1125,0,30.6,4.79,4.83,4.93,4.87,5.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,416,416,https://www.airbnb.com/rooms/1277527400878120655,2024-11-05,city scrape,Experience Historic Albany Full Throttle!,nice,https://a0.muscache.com/pictures/hosting/Hosti...,https://www.airbnb.com/users/show/382970529,Thomas,...,1.4,2.0,365,1,0.0,4.90,4.90,5.00,4.80,2.20
417,417,417,https://www.airbnb.com/rooms/1277717926531215616,2024-11-05,city scrape,Large Center Sq Luxury Retreat.,nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/65760,Anthony,...,3.0,1.7,1125,0,0.0,4.90,4.90,5.00,4.80,2.20
418,418,418,https://www.airbnb.com/rooms/1279011643102137684,2024-11-05,city scrape,nice cool stays,nice,https://a0.muscache.com/pictures/hosting/Hosti...,https://www.airbnb.com/users/show/539608284,Joyhefsiba,...,1.0,4.0,365,1,0.0,4.90,4.90,5.00,4.80,2.20
419,419,419,https://www.airbnb.com/rooms/1279781671588026640,2024-11-05,city scrape,Spacious & Quiet Townhouse.,nice,https://a0.muscache.com/pictures/hosting/Hosti...,https://www.airbnb.com/users/show/528617680,Paola,...,1.0,1.0,365,1,1.0,4.90,4.90,4.90,4.80,1.00


_accommodates_

In [161]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['accommodates'])
unico

array([1., 2., 3., 4., 5., 6., 7.])

In [162]:
df['accommodates'] = df['accommodates'].replace([1, 2, 2.9], 0).replace([3, 4, 5, 6, 7], 1)
df

,Unnamed: 0.1,Unnamed: 0,listing_url,last_scraped,source,name,neighborhood_overview,picture_url,host_url,host_name,...,beds,minimum_nights,maximum_nights,availability_365,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,reviews_per_month
0,0,0,https://www.airbnb.com/rooms/2992450,2024-11-05,city scrape,Luxury 2 bedroom apartment,nice,https://a0.muscache.com/pictures/44627226/0e72...,https://www.airbnb.com/users/show/4621559,Kenneth,...,2.0,1.7,1125,0,9.0,4.90,4.90,4.90,4.80,0.07
1,1,1,https://www.airbnb.com/rooms/3820211,2024-11-05,previous scrape,Restored Precinct in Center Sq. w/Parking,"Great restaurants, architecture, walking, peop...",https://a0.muscache.com/pictures/prohost-api/H...,https://www.airbnb.com/users/show/19648678,Terra,...,1.0,2.0,1125,0,30.6,4.75,4.87,4.81,4.82,2.46
2,2,2,https://www.airbnb.com/rooms/5651579,2024-11-05,city scrape,Large studio apt by Capital Center & ESP@,The neighborhood is very eclectic. We have a v...,https://a0.muscache.com/pictures/b3fc42f3-6e5e...,https://www.airbnb.com/users/show/29288920,Gregg,...,1.0,2.0,45,0,30.6,4.51,4.45,4.87,4.76,3.20
3,3,3,https://www.airbnb.com/rooms/6623339,2024-11-05,city scrape,Center Sq. Loft in Converted Precinct w/ Parking,"Located in Albany's finest urban neighborhood,...",https://a0.muscache.com/pictures/prohost-api/H...,https://www.airbnb.com/users/show/19648678,Terra,...,1.0,2.0,1125,0,30.6,4.73,4.71,4.70,4.80,2.86
4,4,4,https://www.airbnb.com/rooms/9005989,2024-11-05,city scrape,"Studio in The heart of Center SQ, in Albany NY","There are many shops, restaurants, bars, museu...",https://a0.muscache.com/pictures/d242a77e-437c...,https://www.airbnb.com/users/show/17766924,Sugey,...,2.0,1.0,1125,0,30.6,4.79,4.83,4.93,4.87,5.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,416,416,https://www.airbnb.com/rooms/1277527400878120655,2024-11-05,city scrape,Experience Historic Albany Full Throttle!,nice,https://a0.muscache.com/pictures/hosting/Hosti...,https://www.airbnb.com/users/show/382970529,Thomas,...,1.4,2.0,365,1,0.0,4.90,4.90,5.00,4.80,2.20
417,417,417,https://www.airbnb.com/rooms/1277717926531215616,2024-11-05,city scrape,Large Center Sq Luxury Retreat.,nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/65760,Anthony,...,3.0,1.7,1125,0,0.0,4.90,4.90,5.00,4.80,2.20
418,418,418,https://www.airbnb.com/rooms/1279011643102137684,2024-11-05,city scrape,nice cool stays,nice,https://a0.muscache.com/pictures/hosting/Hosti...,https://www.airbnb.com/users/show/539608284,Joyhefsiba,...,1.0,4.0,365,1,0.0,4.90,4.90,5.00,4.80,2.20
419,419,419,https://www.airbnb.com/rooms/1279781671588026640,2024-11-05,city scrape,Spacious & Quiet Townhouse.,nice,https://a0.muscache.com/pictures/hosting/Hosti...,https://www.airbnb.com/users/show/528617680,Paola,...,1.0,1.0,365,1,1.0,4.90,4.90,4.90,4.80,1.00


In [163]:
df['availability_365']

0      0
1      0
2      0
3      0
4      0
      ..
416    1
417    0
418    1
419    1
420    0
Name: availability_365, Length: 421, dtype: int64

**REGRESIÓN LOGÍSTICA**

_host_is_superhost_

In [164]:
#Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep=df[['host_acceptance_rate', 'host_response_rate', 'number_of_reviews']]
Var_Dep = df['host_is_superhost']

In [165]:
X= Vars_Indep
y= Var_Dep

In [166]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=None)

In [167]:
escalar =StandardScaler()

In [168]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [169]:
from sklearn.linear_model import LogisticRegression
algoritma = LogisticRegression()

In [170]:
algoritma.fit(X_train, y_train)

LogisticRegression()

In [171]:
y_pred =algoritma.predict(X_test)
y_pred

array([1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1])

In [172]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[ 34  82]
 [ 23 114]]


In [173]:
#Calcula la prescisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.5816326530612245


In [174]:
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.5849802371541502


In [175]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad =recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.29310344827586204


_host_identity_verified_

In [176]:
#Declaramos las variables dependientes e independientes para regresión logistica
Vars_Indep=df[['has_availability', 'host_response_rate', 'host_acceptance_rate']]
Var_Dep=df['host_identity_verified']

In [177]:
df['host_acceptance_rate'].dtype

dtype('int64')

In [178]:
X1= Vars_Indep
y1= Var_Dep

In [179]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.3, random_state=None)

In [180]:
escalar =StandardScaler()

In [181]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [182]:
from sklearn.linear_model import LogisticRegression
algoritma = LogisticRegression()

In [183]:
algoritma.fit(X_train, y_train)

LogisticRegression()

In [184]:
y_pred =algoritma.predict(X_test)
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [185]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[  0  13]
 [  0 114]]


In [186]:
#Calcula la prescisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.8976377952755905


In [187]:
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.8976377952755905


In [188]:
from sklearn.metrics import recall_score

sensibilidad =recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.0


_host_acceptance_rate_

In [189]:
Vars_Indep= df[["accommodates", "room_type","instant_bookable"]]
Var_Dep= df["host_acceptance_rate"]

In [190]:
X= Vars_Indep
y= Var_Dep

In [191]:
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.3, random_state=None)

In [192]:
escalar= StandardScaler()

In [193]:
X_train= escalar.fit_transform(X_train)
X_test= escalar.fit_transform(X_test)

In [194]:
from sklearn.linear_model import LogisticRegression
algoritmo= LogisticRegression()

In [195]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [196]:
y_pred= algoritmo.predict(X_test)
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [197]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusion: ")
print(matriz)

Matriz de confusion: 
[[  0  21]
 [  0 106]]


In [198]:
from sklearn.metrics import precision_score

precision= precision_score(y_test, y_pred, average="binary", pos_label=1)
print("predicciones del modelo")
print(precision)

predicciones del modelo
0.8346456692913385


In [199]:
from sklearn.metrics import accuracy_score

exactitud= accuracy_score(y_test, y_pred)
print("Exactitud del modelo")
print(exactitud)

Exactitud del modelo
0.8346456692913385


In [200]:
from sklearn.metrics import recall_score

sensibilidad= recall_score(y_test, y_pred, average="binary", pos_label=0)
print("Sensibilidad del modelo: ")
print(sensibilidad)

Sensibilidad del modelo: 
0.0


_has_availability_

In [201]:
Vars_Indep=df[['host_identity_verified', 'host_response_rate', 'host_acceptance_rate']]
Var_Dep=df['has_availability']

In [202]:
df['host_identity_verified'].dtype

dtype('int64')

In [203]:
#Redefinimos las variables
X3 = Vars_Indep
y3 = Var_Dep

In [204]:
X_train, X_test, y_train, y_test = train_test_split(X3, y3, test_size=0.2, random_state=None)

In [205]:
escalar =StandardScaler()

In [206]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [207]:
from sklearn.linear_model import LogisticRegression
algoritma = LogisticRegression()

In [208]:
algoritma.fit(X_train, y_train)

LogisticRegression()

In [209]:
y_pred =algoritma.predict(X_test)
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [210]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[85]]


c:\Users\ady_b\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [211]:
#Calcula la prescisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
1.0


In [212]:
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
1.0


In [213]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad =recall_score(y_test, y_pred, average="binary", pos_label= 0)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.0


c:\Users\ady_b\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


_instant_bookable_

In [214]:
Vars_Indep= df[["accommodates", "room_type"]]
Var_Dep= df["instant_bookable"]

In [215]:
X= Vars_Indep
y= Var_Dep

In [216]:
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.3, random_state=None) 

In [217]:
escalar= StandardScaler()

In [218]:
X_train= escalar.fit_transform(X_train)
X_test= escalar.fit_transform(X_test)

In [219]:
from sklearn.linear_model import LogisticRegression
algoritmo= LogisticRegression()

In [220]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [221]:
y_pred= algoritmo.predict(X_test)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [222]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusion: ")
print(matriz)

Matriz de confusion: 
[[94  0]
 [33  0]]


In [223]:
from sklearn.metrics import precision_score

precision= precision_score(y_test, y_pred, average="binary", pos_label=1)
print("predicciones del modelo")
print(precision)

predicciones del modelo
0.0


c:\Users\ady_b\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [224]:
from sklearn.metrics import accuracy_score

exactitud= accuracy_score(y_test, y_pred)
print("Exactitud del modelo")
print(exactitud)

Exactitud del modelo
0.7401574803149606


In [225]:
from sklearn.metrics import recall_score

sensibilidad= recall_score(y_test, y_pred, average="binary", pos_label=0)
print("Sensibilidad del modelo: ")
print(sensibilidad)

Sensibilidad del modelo: 
1.0


_room_type_

In [226]:
Vars_Indep= df[["accommodates", "room_type", "instant_bookable"]]
Var_Dep= df["room_type"]

In [227]:
X= Vars_Indep
y= Var_Dep

In [228]:
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.3, random_state=None) 

In [229]:
escalar= StandardScaler()

In [230]:
X_train= escalar.fit_transform(X_train)
X_test= escalar.fit_transform(X_test)

In [231]:
from sklearn.linear_model import LogisticRegression
algoritmo= LogisticRegression()

In [232]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [233]:
y_pred= algoritmo.predict(X_test)
y_pred

array([0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1])

In [234]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusion: ")
print(matriz)

Matriz de confusion: 
[[37  0]
 [ 0 90]]


In [235]:
from sklearn.metrics import precision_score

precision= precision_score(y_test, y_pred, average="binary", pos_label=1)
print("predicciones del modelo")
print(precision)

predicciones del modelo
1.0


In [236]:
from sklearn.metrics import accuracy_score

exactitud= accuracy_score(y_test, y_pred)
print("Exactitud del modelo")
print(exactitud)

Exactitud del modelo
1.0


In [237]:
from sklearn.metrics import recall_score

sensibilidad= recall_score(y_test, y_pred, average="binary", pos_label=0)
print("Sensibilidad del modelo: ")
print(sensibilidad)

Sensibilidad del modelo: 
1.0


_accommodates_

In [238]:
Vars_Indep= df[[ "room_type", "instant_bookable"]]
Var_Dep= df["accommodates"]

In [239]:
X= Vars_Indep
y= Var_Dep

In [240]:
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.3, random_state=None) 

In [241]:
escalar= StandardScaler()

In [242]:
X_train= escalar.fit_transform(X_train)
X_test= escalar.fit_transform(X_test)

In [243]:
from sklearn.linear_model import LogisticRegression
algoritmo= LogisticRegression()

In [244]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [245]:
y_pred= algoritmo.predict(X_test)
y_pred

array([1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1.,
       1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1.,
       1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
       1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0.,
       0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1.,
       1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0.,
       1., 1., 1., 1., 1., 0., 1., 1.])

In [246]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusion: ")
print(matriz)

Matriz de confusion: 
[[37 28]
 [ 2 60]]


In [247]:
from sklearn.metrics import precision_score

precision= precision_score(y_test, y_pred, average="binary", pos_label=1)
print("predicciones del modelo")
print(precision)

predicciones del modelo
0.6818181818181818


In [248]:
from sklearn.metrics import accuracy_score

exactitud= accuracy_score(y_test, y_pred)
print("Exactitud del modelo")
print(exactitud)

Exactitud del modelo
0.7637795275590551


In [249]:
from sklearn.metrics import recall_score

sensibilidad= recall_score(y_test, y_pred, average="binary", pos_label=0)
print("Sensibilidad del modelo: ")
print(sensibilidad)

Sensibilidad del modelo: 
0.5692307692307692


_availability_365_

In [250]:
#DEclaramos las variables dependientes e independientes para la regresión logística
Vars_Indep=df[['host_response_rate','price', 'review_scores_location', 'property_type', 'host_identity_verified']]
Var_Dep = df['availability_365']

In [251]:
df['review_scores_cleanliness'].dtype

dtype('float64')

In [252]:
#Redefinimos las variables
X = Vars_Indep
y = Var_Dep

In [253]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = None)

In [254]:
#Se escalan todos los datos
escalar = StandardScaler()

In [255]:
#Para realizar el escalamineto de las variables "X" tanto de entrenamiento como de prueba, utilizaremos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [256]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [257]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [258]:
#REalizamos una prediccion
y_pred = algoritmo.predict(X_test)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [259]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[118   1]
 [  8   0]]


In [260]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.0


In [261]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Excatitud del modelo:')
print(exactitud)

Excatitud del modelo:
0.9291338582677166


In [262]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.9915966386554622


_host_response_time_

In [263]:
#DEclaramos las variables dependientes e independientes para la regresión logística
Vars_Indep=df[['host_verifications', 'host_total_listings_count', 'host_response_rate']]
Var_Dep = df['host_response_time']

In [264]:
df['host_response_time'].dtype
df['host_response_time'].drop_duplicates().tolist()

[0, 1]

In [265]:
#Redefinimos las variables
X = Vars_Indep
y = Var_Dep

In [266]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = None)

In [267]:
#Se escalan todos los datos
escalar = StandardScaler()

In [268]:
#Para realizar el escalamineto de las variables "X" tanto de entrenamiento como de prueba, utilizaremos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [269]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [270]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [271]:
#REalizamos una prediccion
y_pred = algoritmo.predict(X_test)
y_pred

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [272]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[  9  16]
 [  0 102]]


In [273]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.864406779661017


In [274]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Excatitud del modelo:')
print(exactitud)

Excatitud del modelo:
0.8740157480314961


In [275]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.36


_host_has_profile_pic_

In [276]:
#DEclaramos las variables dependientes e independientes para la regresión logística
Vars_Indep=df[['host_verifications', 'review_scores_rating', 'host_total_listings_count', 'host_is_superhost']]
Var_Dep = df['host_has_profile_pic']

In [277]:
df['review_scores_rating'].dtype

dtype('float64')

In [278]:
#Redefinimos las variables
X = Vars_Indep
y = Var_Dep

In [279]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = None)

In [280]:
#Se escalan todos los datos
escalar = StandardScaler()

In [281]:
#Para realizar el escalamineto de las variables "X" tanto de entrenamiento como de prueba, utilizaremos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [282]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [283]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [284]:
#REalizamos una prediccion
y_pred = algoritmo.predict(X_test)
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [285]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[  0   5]
 [  0 122]]


In [286]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.9606299212598425


In [287]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Excatitud del modelo:')
print(exactitud)

Excatitud del modelo:
0.9606299212598425


In [289]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.0
